In [1]:
from __init__ import *
from snmcseq_utils import cd
from snmcseq_utils import create_logger
from CEMBA_update_mysql import connect_sql


In [2]:
log = create_logger()
log.info("Hello")

07/03/2018 02:58:00 PM Hello


In [3]:
# ens = 'Ens10'
# cluster_type = 'cluster_mCHmCG_lv_npc50_k30'
# annotation_type = 'annotation' + cluster_type[len('cluster'):]
# engine = connect_sql('CEMBA')

# # cluster annotation
# sql = """SELECT {}, {} FROM {}""".format(cluster_type, annotation_type, ens)
# df_info = pd.read_sql(sql, engine)
# df_info = df_info.sort_values(cluster_type).drop_duplicates().set_index(cluster_type)

# ens_path = os.path.join(PATH_ENSEMBLES, ens)
# n_clusters = len(glob.glob(os.path.join(ens_path, 'allc_merged/allc_merged_mCG_{}_*_{}.tsv'.format(cluster_type, ens))))

# dmr results
# input_f = os.path.join(ens_path, 'dmr/dmr_allc_merged_mCG_{}_rms_results_collapsed.tsv'.format(cluster_type))
input_f = '/cndd/Public_Datasets/CEMBA/snmCSeq/Ensembles/Ens10/dmr/cgdmr_multimodal_v1_rms_results_collapsed.tsv'
df = pd.read_table(input_f, index_col=['#chr', 'start', 'end'], dtype={'#chr': object})

In [7]:
key = 'hypomethylated_samples'
num_dms = 20 
res = df.loc[(~df[key].isnull()) & (df['number_of_dms'] >= num_dms), :].copy()
res['name'] = np.arange(res.shape[0]) 
print(res.shape)

# with cd(os.path.join(PATH_ENSEMBLES, ens, 'dmr')):
#     res[['name', 'number_of_dms']].to_csv('dmr_allc_merged_mCG_cluster_mCHmCG_lv_npc50_k30_rms_results_collapsed_{}dms.tsv.DMR.bed'.format(num_dms),
#                                      sep='\t', na_rep='NA', header=False, index=True)

(60966, 30)


In [9]:
def dmr_counts(ens, n_clusters, cluster_type, df, num_dms=3, keys=['hypomethylated_samples', 'hypermethylated_samples']):
    """
    cluster_type examples:
    cluster_type = 'cluster_mCHmCG_lv_npc50_k30'
    
    """
    res_out = []
    for key in keys:
        res_key = []
        res = df.loc[(~df[key].isnull()) & (df['number_of_dms'] >= num_dms), key].apply(lambda x: x.split(','))
        for i in range(n_clusters):
            res_cluster = res.loc[
                res.apply(lambda x: ('merged_mCG_{}_{}_{}'.format(cluster_type, i+1, ens) in x))]
            res_key.append({cluster_type: i+1, 'n_dmr_{}'.format(key[:4]): res_cluster.shape[0]})
        res_key = pd.DataFrame(res_key).set_index(cluster_type)
        res_out.append(res_key)
    return pd.concat(res_out, axis=1)

In [10]:
df_ndmr = dmr_counts(ens, n_clusters, cluster_type, df, num_dms=20)
df_ndmr = pd.merge(df_info, df_ndmr, left_index=True, right_index=True)

In [11]:
df_ndmr

annotation_mCHmCG_lv_npc50_k30  n_dmr_hypo  \
cluster_mCHmCG_lv_npc50_k30                                              
1                                                      mL4       27682   
2                                                    mL2/3       31139   
3                                                    mL2/3       30662   
4                                                    mL5-1       19799   
5                                                    mL6-2       21527   
6                                                    mL5-1       18382   
7                                                    mDL-2       10904   
8                                                    mDL-2        9609   
9                                                   mSst-1        7320   
10                                                   mL5-2       13916   
11                                                     mPv       11319   
12                                                   mL6-2       14551   
13                                                   mL6-2       11466   
14                                                   mL6-2       11696   
15                                                    mVip        9202   
16                                                   mL6-1        3441   
17                                                 mNdnf-2       14997   
18                                                    None       12463   
19                                                     mL4        4366   
20                                                   mL6-2        8429   
21                                                    None       10664   
22                                                   mL6-2        2245   
23                                                    None       11291   
24                                                     mPv        6061   
25                                                  mSst-1        3161   
26                                                   mDL-3        1455   

                             n_dmr_hype  
cluster_mCHmCG_lv_npc50_k30              
1                                 10068  
2                                 10067  
3                                  5573  
4                                  7061  
5                                 13656  
6                                  4337  
7                                  3393  
8                                  6185  
9                                 33769  
10                                 6208  
11                                32667  
12                                 4827  
13                                11987  
14                                 6011  
15                                30669  
16                                27783  
17                                19278  
18                                32748  
19                                 7399  
20                                 4087  
21                                29811  
22                                11985  
23                                23704  
24                                17171  
25                                 8974  
26                                 1368

In [4]:
print(df.shape)
df_hypo = df.loc[(df['number_of_dms']>=3) & (~df['hypomethylated_samples'].isnull()), 'hypomethylated_samples'].apply(lambda x: x.split(','))
df_hyper = df.loc[(df['number_of_dms']>=3) & (~df['hypermethylated_samples'].isnull()), 'hypermethylated_samples'].apply(lambda x: x.split(','))

print(df_hypo.shape)
print(df_hyper.shape)

(981451, 44)
(437526,)
(281881,)


In [5]:
df_hypo.head()
# df_hypo_cluster = df_hypo.loc[
#         df_hypo.apply(lambda x: ('merged_mCG_{}_{}_{}'.format(cluster_type, i+1, ens) in x))]

#chr  start    end    
1     3044313  3044403    [multimodal_v1_clst1, multimodal_v1_clst23, mu...
      3100308  3100351           [multimodal_v1_clst9, multimodal_v1_clst3]
      3113767  3114036           [multimodal_v1_clst1, multimodal_v1_clst5]
      3118791  3118936    [multimodal_v1_clst26, multimodal_v1_clst4, mu...
      3130148  3130281                                [multimodal_v1_clst1]
Name: hypomethylated_samples, dtype: object

In [11]:
outdir = '/cndd/Public_Datasets/CEMBA/snmCSeq/Ensembles/Ens10/dmr/cgdmr_multimodal_v1'
if not os.path.isdir(outdir):
    os.makedirs(outdir)
    logging.info('Created directory {}'.format(outdir))
    
clsts = [i[len('methylation_level_'):]for i in df.filter(regex='^methylation_level_*').columns]
for clst in clsts:
    df_hypo_cluster = df_hypo[df_hypo.apply(lambda x: (clst in x))]
    output = os.path.join(outdir, '{}.bed'.format(clst))
    df_hypo_cluster.to_csv(output, sep='\t', header=False, index=True, na_rep='NA')
    logging.info("Saved to {}".format(output))
    logging.info(df_hypo_cluster.shape)

07/03/2018 03:11:48 PM Saved to /cndd/Public_Datasets/CEMBA/snmCSeq/Ensembles/Ens10/dmr/cgdmr_multimodal_v1/multimodal_v1_clst39.bed
07/03/2018 03:11:48 PM (4888,)
07/03/2018 03:11:50 PM Saved to /cndd/Public_Datasets/CEMBA/snmCSeq/Ensembles/Ens10/dmr/cgdmr_multimodal_v1/multimodal_v1_clst37.bed
07/03/2018 03:11:50 PM (8017,)
07/03/2018 03:11:52 PM Saved to /cndd/Public_Datasets/CEMBA/snmCSeq/Ensembles/Ens10/dmr/cgdmr_multimodal_v1/multimodal_v1_clst30.bed
07/03/2018 03:11:52 PM (32687,)
07/03/2018 03:11:54 PM Saved to /cndd/Public_Datasets/CEMBA/snmCSeq/Ensembles/Ens10/dmr/cgdmr_multimodal_v1/multimodal_v1_clst21.bed
07/03/2018 03:11:54 PM (13537,)
07/03/2018 03:11:56 PM Saved to /cndd/Public_Datasets/CEMBA/snmCSeq/Ensembles/Ens10/dmr/cgdmr_multimodal_v1/multimodal_v1_clst26.bed
07/03/2018 03:11:56 PM (26144,)
07/03/2018 03:11:59 PM Saved to /cndd/Public_Datasets/CEMBA/snmCSeq/Ensembles/Ens10/dmr/cgdmr_multimodal_v1/multimodal_v1_clst28.bed
07/03/2018 03:11:59 PM (23856,)
07/03/2018 0

In [21]:
with cd(os.path.join(PATH_ENSEMBLES, ens, 'dmr')):
    f = 'dmr_allc_merged_mCG_cluster_mCHmCG_lv_npc50_k30_rms_results_collapsed.tsv.DMR.bed'
    df_bed = pd.read_table(f, header=None, names=['chr', 'start', 'end', 'num_dms'])
print(df_bed.shape)
df_bed.head()

(1162175, 4)


chr    start      end  num_dms
0  chr1  3013472  3013473        1
1  chr1  3023074  3023075        1
2  chr1  3027397  3027398        1
3  chr1  3034471  3034472        1
4  chr1  3044312  3044403        3

In [23]:
df_out = df_bed[df_bed['num_dms']>=20]
with cd(os.path.join(PATH_ENSEMBLES, ens, 'dmr')):
    df_out.to_csv('dmr_allc_merged_mCG_cluster_mCHmCG_lv_npc50_k30_rms_results_collapsed_20dms.tsv.DMR.bed', 
                 sep='\t', na_rep='NA', header=False, index=False)